In [184]:
import os
import sys
import chardet


def read_text(path: str) -> str | None:
    try:
        if not os.path.exists(path):
            return None

        with open(path, "rb") as file:
            data = file.read()

        encoding = chardet.detect(data)["encoding"]
        return data.decode(encoding)
    except Exception as e:
        return None

In [185]:
import regex as re

# 4 digits with no other digits before or after
reg = r'(?<!\d)\d{4}(?!\d)'
def get_year(file_name: str) -> int|None:
    if re.search(reg, file_name) is not None:
        return int(re.search(reg, file_name).group(0))
    else:
        return None
    

def count_regexp_occ(regexp: str, text: str) -> int:
    return len(re.findall(regexp, text))


In [186]:
regex你好 = r'((?<!\p{Han})(你|您)好[啊]?(?!\p{Han}))'
regex嗨 = r'((?<!\p{Han})(嗨|嘿)(?!\p{Han}))'
regex上午下午晚上中午早上好 = r'((?<!\p{Han})(上午|下午|晚上|中午|早上)好[啊]?(?!\p{Han}))'
regex你好吗 = r'((?<!\p{Han})你(最近)?好[吗么没嘛啊](?!\p{Han}))'
regex吃了吗 = r'(吃[过]?[饭]?了[吗么没嘛啊](?!\p{Han}))'

all_regex = [regex你好, regex嗨, regex上午下午晚上中午早上好, regex你好吗, regex吃了吗]

In [187]:

test_text = "早上好。我是为了你好，你好,你好你好-嗨！-嘿！你吃过饭了吗？你吃了吗？吃了没？你好吗？你最近好吗？你说我好吗？"
result = re.findall(pattern=regex嗨, string=test_text)
print(result)
result = re.findall(pattern=regex吃了吗, string=test_text)
print(result)
result = re.findall(pattern=regex你好吗, string=test_text)
print(result)
result = re.findall(pattern=regex上午下午晚上中午早上好, string=test_text)
print(result)

[('嗨', '嗨'), ('嘿', '嘿')]
['吃过饭了吗', '吃了吗', '吃了没']
[('你好吗', ''), ('你最近好吗', '最近')]
[('早上好', '早上')]


In [188]:
def walk_decade(decade: int):
    for root, dirs, files in os.walk(f"./captions/{decade}s"):
        for file in files:
            path = os.path.join(root, file)
            text = read_text(path)
            if text is None:
                continue

            year_of_movie = get_year(file)
            
            counts = [
                count_regexp_occ(r, text)
                for r in all_regex
            ]

            yield file, decade, str(year_of_movie), counts[0], counts[1], counts[2], counts[3], counts[4]

In [189]:
import pandas as pd

In [190]:
def generate_csv(decade: int):
    data = list(walk_decade(decade))

    df = pd.DataFrame(data, columns=[
        "file", "decade", "year",
            "regex你好", 
            "regex嗨",
            "regex上午下午晚上中午早上好",
            "regex你好吗",
            "regex吃了吗",
    ])
    df.to_csv(f"{decade}.csv", index=False)

In [191]:
for y in range(1960, 2030, 10):
    print(y)
    generate_csv(y)

1960
1970
1980
1990
2000
2010
2020


In [192]:
#join all csvs
import pandas as pd

df = pd.read_csv("1960.csv")

for y in range(1970, 2030, 10):
    df = df.append(pd.read_csv(f"{y}.csv"))
    


/var/folders/gx/j0snwyqn015d2ms_6k_qbspc0000gn/T/ipykernel_64278/236019991.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(f"{y}.csv"))
/var/folders/gx/j0snwyqn015d2ms_6k_qbspc0000gn/T/ipykernel_64278/236019991.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(f"{y}.csv"))
/var/folders/gx/j0snwyqn015d2ms_6k_qbspc0000gn/T/ipykernel_64278/236019991.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.read_csv(f"{y}.csv"))
/var/folders/gx/j0snwyqn015d2ms_6k_qbspc0000gn/T/ipykernel_64278/236019991.py:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(p

In [193]:
# df["real_year"] = df["year"] is "None" ? df['decade'] : ((df["year"]//10)*10)
df["real_year"] = df.apply(
    lambda row: int(row["decade"]) if row['year'] == "None" else int(row["year"]), axis=1).apply(
    lambda x: int(x//10)*10)
    


In [194]:
df.to_csv("all.csv", index=False)